In [1]:
import numpy as np
import tensorflow as tf

In [2]:
def synthetic_data(w, b, num_examples):
    X = tf.zeros((num_examples, w.shape[0]))
    X += tf.random.normal(shape=X.shape)
    y = tf.matmul(X, tf.reshape(w, (-1, 1))) + b
    y += tf.random.normal(shape=y.shape, stddev=0.01)
    y = tf.reshape(y, (-1, 1))
    return X, y

In [3]:
true_w = tf.constant([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [4]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = tf.data.Dataset.from_tensor_slices(data_arrays)
    if is_train:
        datasets = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [5]:
next(iter(data_iter))

(<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
 array([[-6.1079293e-02, -1.7131927e+00],
        [-1.2024800e+00, -6.7693770e-02],
        [-7.7521211e-01,  6.7655897e-01],
        [ 1.5863883e+00, -9.8200011e-01],
        [ 6.5017396e-01, -2.2809087e-01],
        [-4.7427580e-01,  3.3148313e-01],
        [-1.0175009e+00,  2.4210582e+00],
        [ 9.8362017e-01,  2.0798836e+00],
        [ 2.1622427e-02,  5.5003708e-04],
        [ 6.5466642e-01,  1.0743842e+00]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[ 9.898133  ],
        [ 2.0242906 ],
        [ 0.34319374],
        [10.718055  ],
        [ 6.268248  ],
        [ 2.1240997 ],
        [-6.072168  ],
        [-0.90901315],
        [ 4.2369723 ],
        [ 1.8694717 ]], dtype=float32)>)

In [6]:
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1))

In [7]:
initializer = tf.initializers.RandomNormal(stddev=0.01)
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1, kernel_initializer=initializer))

In [8]:
loss = tf.keras.losses.MeanSquaredError()

In [9]:
trainer = tf.keras.optimizers.SGD(learning_rate=0.03)

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for x, y in data_iter:
        with tf.GradientTape() as tape:
            l = loss(net(x, training=True), y)
        grads = tape.gradient(l, net.trainable_variables)
        trainer.apply_gradients(zip(grads, net.trainable_variables))
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000186
epoch 2, loss 0.000105
epoch 3, loss 0.000105


In [11]:
w = net.get_weights()[0]
print('w的估计误差：', true_w - tf.reshape(w, true_w.shape))
b = net.get_weights()[1]
print('b的估计误差：', true_b - b)

w的估计误差： tf.Tensor([-0.00010824 -0.00071955], shape=(2,), dtype=float32)
b的估计误差： [-0.00027084]
